## Investigate UCI retail data

this notebook investigates the dataset from UCI, and summarizes key parts of it, that should be handled in data cleaning
(it assumes a time series forecasting project will be conducted on the data)

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path().resolve().parent))

import os
import sqlite3
import pandas as pd
from dotenv import load_dotenv
from src.data_processing.data_loader import DataLoader
from src.data_processing.data_cleaner import DataCleaner

load_dotenv()
DB_FILE = os.getenv('DB_FILE')
DB_INPUT_TABLE_NAME = os.getenv('DB_INPUT_TABLE_NAME')
DB_OUTPUT_TABLE_NAME = os.getenv('DB_OUTPUT_TABLE_NAME')

In [ ]:
df = pd.read_parquet('/Users/soo/repos/private/uci_online_retail/data/pipeline_runs/cleaned_data.parquet')
from src.data_processing.data_splitter import DataSplitter
from src.log_config import setup_logging
setup_logging()
data_splitter = DataSplitter()
train, test, features = data_splitter.run(
    df=df,
    date_column='InvoiceDate',
    target_column='Quantity',
    days_in_test_split=7
)

# print(f'Train first and last days: {train["InvoiceDate"].min()} - {train["InvoiceDate"].max()}')
# print(f'Test first and last days: {test["InvoiceDate"].min()} - {test["InvoiceDate"].max()}')
# print(f'Feature first and last days: {features["InvoiceDate"].min()} - {features["InvoiceDate"].max()}')

# print(f'all dates in test: {test["InvoiceDate"].unique()}')
# print(f'all dates in features: {features["InvoiceDate"].unique()}')
# print(f'all dates in train: {train["InvoiceDate"].unique()}')

### Run data cleaner class

findings:


InvoiceNo values containing letters (other than 'C' for cancelled) suggests these are not proper sales transactions.


StockCode could should be a 5-digit integral number uniquely assigned to each product. 
It contains values with letters as well. 

e.g. 
- POST = Postage
- DOT = DOTCOM POSTAGE
- M = MANUAL
- PADS = Pads to match all cushions
- S = samples
- gift_<> = Gift voucher
- C2 = CARRIAGE
- BANK CHARGES = Bank Charges
- B = Adjust bad debt
- AMAZONFEE = AMAZON FEE



In [ ]:
with DataCleaner(db_path=DB_FILE) as cleaner:
    exploration_df = cleaner.run(source_table=DB_INPUT_TABLE_NAME, target_table='exploration', countries=['United Kingdom'])

exploration_df.shape

In [ ]:
# summary tables

exploration_df['revenue'] = exploration_df['Quantity'] * exploration_df['UnitPrice']
# count of articles, mean(quantity, revenue and price) and std-dev(quantity, revenue and price) 
summary_df = exploration_df.agg({
    'InvoiceNo': 'nunique',
    'StockCode': 'nunique',
    'Quantity': ['mean', 'std'],
    'UnitPrice': ['mean', 'std'],
    'revenue': ['mean', 'std']
})

most_sold = exploration_df.groupby('Description').agg({'Quantity': 'sum', 'revenue': 'sum'})

exploration_df['date'] = pd.to_datetime(exploration_df['InvoiceDate']).dt.date
bussiest_days = exploration_df.groupby('date').agg({'Quantity': 'sum', 'revenue': 'sum'})



import matplotlib.pyplot as plt
ax = bussiest_days['revenue'].plot(figsize=(12,6), color='blue', label='Revenue')
bussiest_days['Quantity'].plot(figsize=(12,6), secondary_y=True, color='orange', ax=ax, alpha=0.7, label='Quantity')
ax.set_ylabel('Revenue')
ax.right_ax.set_ylabel('Quantity')

# Combine legends from both axes
lines1, labels1 = ax.get_legend_handles_labels()
lines2, labels2 = ax.right_ax.get_legend_handles_labels()
ax.legend(lines1 + lines2, labels1 + labels2, loc='upper left')

plt.xlabel('Date')
plt.title(f"Daily Revenue and Quantity Sold. Correlation: {bussiest_days['revenue'].corr(bussiest_days['Quantity']):.2f}")
plt.show()

In [ ]:
# 